# 01-OpenAI的API调用

! pip install openai langchain-openai

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

llm.invoke("中国的首都是哪里?")


AIMessage(content='中国的首都是北京。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 16, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_808245b034', 'finish_reason': 'stop', 'logprobs': None}, id='run-c29dd9c4-2312-4fae-ae30-7eadc3019308-0', usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}})

## 基本示例: 提示+模型+输出解析器

最基本和常见的用例是将提示模板和模型链接在一起。为了看看这是如何工作的，让我们创建一个接受主题并生成小红书短文的链：

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("请帮我生成一篇关于{topic}的小红书短文")
model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "胖东来烤鸭"})


'标题：胖东来烤鸭，让味蕾沉醉的美味之旅\n\n在繁华的都市中，总是有一些小店铺，隐藏着让人心驰神往的美食。而胖东来烤鸭就是其中一家，它位于街角的小巷里，不起眼却让人流连忘返。\n\n走进胖东来烤鸭，首先映入眼帘的是一股浓浓的烤鸭香味，让人垂涎欲滴。店内装修简洁大方，一排排烤鸭挂在墙上，金黄诱人。坐下来，服务员热情地为你端上一份烤鸭，外酥内嫩，肉质鲜美，让人忍不住一口接一口。\n\n除了烤鸭，胖东来还有许多其他美味的小吃，比如酥皮鲜肉包、炸酱面等等。每一样都是店家精心制作的，口感十分独特。而且价格实惠，让人可以尽情享受美食的同时不心疼钱包。\n\n胖东来烤鸭，不仅是一家小店铺，更是一段让味蕾沉醉的美味之旅。每一次光顾，都会让人心情愉悦，口福满满。如果你也是美食爱好者，不妨来胖东来烤鸭品尝一番，相信你也会被它的魅力所吸引。'

## Prompt提示

prompt 是一个 BasePromptTemplate ，这意味着它接受模板变量的字典并生成一个 PromptValue 。 PromptValue 是一个完整提示的包装器，可以传递给 LLM （它接受一个字符串作为输入）或 ChatModel （它接受一个序列作为输入的消息）。它可以与任何一种语言模型类型一起使用，因为它定义了生成 BaseMessage 和生成字符串的逻辑。

In [6]:
prompt = ChatPromptTemplate.from_template("请根据下面的主题，帮我生成一篇小红书营销短文: {topic}")
prompt_value = prompt.invoke({"topic": "康师傅绿茶"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='请根据下面的主题，帮我生成一篇小红书营销短文: 康师傅绿茶', additional_kwargs={}, response_metadata={})])

## Model模型

Prompt_value被传给LLM，在本例中我们的LLM是ChatModel，意味着它将输BaseMessage

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content='标题：尝鲜康师傅绿茶，清爽一夏\n\n夏日炎炎，一杯清凉的绿茶是最好的解暑良方。康师傅绿茶，清新爽口，让你在炎炎夏日中保持清爽。\n\n康师傅绿茶选用上等绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，清香扑鼻，回味无穷。每一口都是清凉的享受，让你感受到清爽的味蕾刺激。\n\n不仅如此，康师傅绿茶还添加了多种维生素和矿物质，补充身体所需的营养，让你在享受清凉的同时，也能保持健康。无糖低热量的配方，更是适合各个年龄段的人群，让你无负担地畅饮一夏。\n\n快来尝鲜康师傅绿茶，清爽一夏，让清凉伴随你的每一天！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 358, 'prompt_tokens': 47, 'total_tokens': 405, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_808245b034', 'finish_reason': 'stop', 'logprobs': None}, id='run-8613d3bc-3727-43c4-aae6-087e7bdc34cb-0', usage_metadata={'input_tokens': 47, 'output_tokens': 358, 'total_tokens': 405, 'input_token_details': {}, 'output_token_details': {}})

## Output Parser输出解析器

将 model 输出传递给 output_parser ，这是一个 BaseOutputParser ，意味着它接受字符串或 BaseMessage 作为输入。 StrOutputParser 特别简单地将任何输入转换为字符串。

input ->(dict) -> PromptTemplate -> (prompt_value) -> ChatModel -> (ChatMessage) -> OutputParser -> (str) -> result

In [8]:
response = StrOutputParser().invoke(message)
response


'标题：尝鲜康师傅绿茶，清爽一夏\n\n夏日炎炎，一杯清凉的绿茶是最好的解暑良方。康师傅绿茶，清新爽口，让你在炎炎夏日中保持清爽。\n\n康师傅绿茶选用上等绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，清香扑鼻，回味无穷。每一口都是清凉的享受，让你感受到清爽的味蕾刺激。\n\n不仅如此，康师傅绿茶还添加了多种维生素和矿物质，补充身体所需的营养，让你在享受清凉的同时，也能保持健康。无糖低热量的配方，更是适合各个年龄段的人群，让你无负担地畅饮一夏。\n\n快来尝鲜康师傅绿茶，清爽一夏，让清凉伴随你的每一天！'

## 流式输出

如果我们想流式传输结果，我们需要更改我们的函数，使其返回一个生成器。通过for循环打印输出效果

In [12]:

chain = prompt | model | StrOutputParser()

for chunk in chain.stream({"topic": "康师傅绿茶"}):
    print(chunk, end="", flush=True)


标题：探寻康师傅绿茶的清新之美

大家好，今天我要和大家分享的是我最近发现的一款清新美味的饮品——康师傅绿茶。作为一名资深茶饮爱好者，我对绿茶的要求可谓是相当高，而康师傅的这款绿茶完全符合了我的口味。

首先，康师傅绿茶选用的是优质绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，清香扑鼻，让人一口喝下去就仿佛置身于茶园之中。而且，康师傅绿茶不含防腐剂和人工色素，喝起来更加放心和健康。

其次，康师傅绿茶口感清爽，不会过于苦涩或过甜，恰到好处的甜度和苦涩度让人回味无穷。无论是在炎炎夏日还是在寒冷冬季，一瓶康师傅绿茶都能为你带来清凉的享受。

最后，康师傅绿茶的包装设计简约大方，瓶身透明，让人一眼就能看到清澈的茶水，让人对其质量放心。而且，瓶盖设计独特，方便携带，无论是上班还是出行，都能随时享受到康师傅绿茶的清新美味。

总的来说，康师傅绿茶是一款清新美味的饮品，无论是口感还是包装设计都能让人满意。如果你也是绿茶爱好者，不妨试试康师傅绿茶，相信你会爱上它的清新之美。让我们一起探寻康师傅绿茶的独特魅力吧！#康师傅绿茶#清新美味#茶饮爱好者#小红书#营销短文